In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

/var/folders/9d/wpysbld95rg17k1z5kgx_jfh0000gn/T/ipykernel_46429/692205595.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
data = pd.read_csv("datasets/dataset_2.csv")

In [4]:
data.head()

,Name,Trips Completed,Accept Rate,Supply Hours,Rating
0,Abdul,1,100%,3,4.8
1,Abraham,12,83%,5,4.7
2,Adelina,1,100%,2,4.7
3,Akilah,1,100%,2,4.9
4,Alec,21,76%,11,5.0


In [5]:
data.describe()

,Trips Completed,Supply Hours,Rating
count,119.000000,119.000000,119.000000
mean,11.941176,7.084034,4.698571
std,5.857480,2.794000,0.406876
min,1.000000,1.000000,1.600000
25%,8.000000,5.000000,4.600000
50%,12.000000,7.000000,4.800000
75%,15.000000,9.000000,4.900000
max,31.000000,11.000000,5.000000


In [6]:
data.describe(include=['object'])

,Name,Accept Rate
count,119,119
unique,119,39
top,Abdul,100%
freq,1,47


In [7]:
data['numerical_percentage'] = data['Accept Rate'].str.rstrip('%').astype('float')

In [8]:
data.head()

,Name,Trips Completed,Accept Rate,Supply Hours,Rating,numerical_percentage
0,Abdul,1,100%,3,4.8,100.0
1,Abraham,12,83%,5,4.7,83.0
2,Adelina,1,100%,2,4.7,100.0
3,Akilah,1,100%,2,4.9,100.0
4,Alec,21,76%,11,5.0,76.0


### Option 1: $50 for each driver that is online at least 8 hours, accepts 90% of requests, completes 10 trips, and has a rating of 4.7 or better during the time frame;

In [9]:
drivers_8_hours = data[data['Supply Hours'] >= 8]

In [11]:
drivers_8_hours.describe()

,Trips Completed,Supply Hours,Rating,numerical_percentage
count,55.000000,55.000000,55.000000,55.000000
mean,15.545455,9.618182,4.805455,86.836364
std,3.924369,1.146507,0.186009,14.286440
min,8.000000,8.000000,4.300000,44.000000
25%,13.000000,9.000000,4.700000,83.000000
50%,15.000000,10.000000,4.900000,90.000000
75%,18.000000,11.000000,4.950000,100.000000
max,24.000000,11.000000,5.000000,100.000000


As we can see, there are 55 drivers who worked more than 8 hours. Let's check all the other conditions.

In [14]:
driver_90_percent = drivers_8_hours[drivers_8_hours['numerical_percentage'] >= 90]

In [15]:
option_1_drivers = driver_90_percent[(driver_90_percent['Rating'] >= 4.7) & (driver_90_percent['Trips Completed'] >= 10)]

In [16]:
option_1_drivers.head()

,Name,Trips Completed,Accept Rate,Supply Hours,Rating,numerical_percentage
11,Byron,15,100%,10,4.9,100.0
12,Carlota,14,100%,8,5.0,100.0
19,Dannette,14,100%,9,4.9,100.0
23,Demetrius,14,100%,9,5.0,100.0
26,Dimple,15,100%,10,4.9,100.0


In [17]:
option_1_drivers.describe()

,Trips Completed,Supply Hours,Rating,numerical_percentage
count,21.000000,21.000000,21.000000,21.000000
mean,14.714286,9.571429,4.919048,96.952381
std,3.211586,1.075706,0.074960,3.866215
min,10.000000,8.000000,4.800000,90.000000
25%,13.000000,9.000000,4.900000,94.000000
50%,14.000000,10.000000,4.900000,100.000000
75%,15.000000,10.000000,5.000000,100.000000
max,23.000000,11.000000,5.000000,100.000000


So according to option 1, we'd have to pay out 21 x 50 = $1050

Then we have option 2, which is
### Option 2: $4/trip for all drivers who complete 12 trips, and have a 4.7 or better rating. //
Let's calculate how many drivers that is

In [18]:
option_2_drivers = data[(data['Rating'] >= 4.7) & (data['Trips Completed'] >= 12)]

In [19]:
option_2_drivers.describe()

,Trips Completed,Supply Hours,Rating,numerical_percentage
count,46.000000,46.000000,46.000000,46.000000
mean,16.173913,9.260870,4.893478,81.217391
std,3.719644,1.638899,0.095224,19.620752
min,12.000000,5.000000,4.700000,19.000000
25%,13.250000,9.000000,4.800000,72.000000
50%,15.000000,9.500000,4.900000,86.000000
75%,18.750000,11.000000,5.000000,99.000000
max,26.000000,11.000000,5.000000,100.000000


According to option 2, we have 46 drivers. Let's calculate the payout.

In [21]:
payout_option_2 = sum(option_2_drivers['Trips Completed'])
print(f"The payout for option 2 is {payout_option_2 * 4}")

The payout for option 2 is 2976


So we see that the payout for option 2 is significantly higher than that for option 1.

### How many drivers would qualify for a bonus under Option 1 but not under Option 2?

To calculate how many drivers would qualify under option 1 but not under 2, we'll have to remove the drivers from option 1 who also qualify for option 2. 

In [22]:
# Merge dataframes with indicator
merged_df = pd.merge(option_1_drivers, option_2_drivers, on='Name', how='left', indicator=True)

# Filter out the common entries
result_df = merged_df[merged_df['_merge'] == 'left_only'].drop(columns=['_merge'])

In [23]:
result_df.head()

,Name,Trips Completed_x,Accept Rate_x,Supply Hours_x,Rating_x,numerical_percentage_x,Trips Completed_y,Accept Rate_y,Supply Hours_y,Rating_y,numerical_percentage_y
16,Oren,11,91%,9,4.8,91.0,NaN,NaN,NaN,NaN,NaN
17,Phyllis,10,90%,8,4.8,90.0,NaN,NaN,NaN,NaN,NaN


In [24]:
result_df.describe()

,Trips Completed_x,Supply Hours_x,Rating_x,numerical_percentage_x,Trips Completed_y,Supply Hours_y,Rating_y,numerical_percentage_y
count,2.000000,2.000000,2.0,2.000000,0.0,0.0,0.0,0.0
mean,10.500000,8.500000,4.8,90.500000,NaN,NaN,NaN,NaN
std,0.707107,0.707107,0.0,0.707107,NaN,NaN,NaN,NaN
min,10.000000,8.000000,4.8,90.000000,NaN,NaN,NaN,NaN
25%,10.250000,8.250000,4.8,90.250000,NaN,NaN,NaN,NaN
50%,10.500000,8.500000,4.8,90.500000,NaN,NaN,NaN,NaN
75%,10.750000,8.750000,4.8,90.750000,NaN,NaN,NaN,NaN
max,11.000000,9.000000,4.8,91.000000,NaN,NaN,NaN,NaN


There are only two drivers who qualify for option 1 but not for option 2

### What percentages of drivers online completed less than 10 trips, had an acceptance rate of less than 90%, and had a rating of 4.7 or higher?

In [26]:
question_4_drivers = data[(data['Trips Completed'] >= 10) & (data["numerical_percentage"] < 90) & (data['Rating'] >= 4.7)]

In [28]:
print(f'The percentage of drivers described in the question is {len(question_4_drivers) * 100 / len(data)}')

The percentage of drivers described in the question is 24.369747899159663


### Scenario 2: A taxi driver currently generates $200 per day in fares (before expenses), works six days a week, takes three weeks off, and has the following expenses:

Gas - $200 per week
Insurance - $400 per month
Vehicle rent (by the week) - $500
The driver doesn't pay gas and rent expenses on off weeks.

Now, let's assume that the same driver would buy a Town Car and partner with Uber. If he does, his gas expenses would go up by 5%, his insurance expense would decrease by 20%, and he would no longer be renting a vehicle. However, he would need to buy a car. The driver would still take three weeks off per year.

Given Scenario 2, provide answers to the questions below:

### How much money (after expenses) does the taxi driver make per year without partnering with Uber?

This is his weekly income - 200 * 6 - 200 (gas) - 100 (insurange) - 500 (vehicle) = 400
And he works 52 - 3 = 49 weeks.
Thus, his yearly income is 49 * 400 = $19600


### You are convincing the same driver above to buy a Town Car and partner with Uber. Assuming the new car is 40,000 USD, how much would the driver's gross fares need to increase per week to fully pay for the car in year 1 and maintain the same yearly profit margin as before?

For him to cover the cost of the car in 1 year and maintain the same yearly profit margin, he'd have to earn 40000 + 19600 = 59600usd in a year.

His weekly earnings/expenses after the switch to Uber is this:
x (earning) - 200 x 105% (gas) - 100 x 80% (insurance) 
 = x - 210 - 80 = x - 290 usd per week.

 Considering that he still works the same days and same number of weeks, 49(x-290) = 59600
 So his weekly earnings would have to be ~= 1507usd, and his daily earnings ~=252 usd, which is not all that much more than his current earnings considering he would own a car.